In [1]:
import pymongo
import pandas as pd
import numpy as np
import datetime
import os
import collections
import time

In [2]:
client = pymongo.MongoClient()
db = client['Fields']
path = r'C:\Users\ChastLai\Documents\Data\Ticks\\'
tick_list = os.listdir(path)
data = pd.DataFrame()
price = pd.DataFrame(db['收盤價'].find({'日期': {'$gte': datetime.datetime(2022, 9, 1), '$lte': datetime.datetime(2022, 11, 7)}}, {'_id': 0})).set_index('日期')
priceo = pd.DataFrame(db['開盤價'].find({'日期': {'$gte': datetime.datetime(2022, 9, 1), '$lte': datetime.datetime(2022, 11, 7)}}, {'_id': 0})).set_index('日期')
price = (price / priceo - 1).shift(-1)
price.columns = price.columns.astype(int)

In [3]:
total_, iso_, nis_, nis_s_, nis_c_, nis_b_ = [], [], [], [], [], []
df_ = pd.DataFrame()
def calculator(df_, tick):
    ctime = time.time()
    stk_test = pd.read_json(path + tick)
    stk_test['ts'] = pd.to_datetime(stk_test['ts'])
    stk_test.set_index('ts', inplace=True)
    pf = price.loc[stk_test.index[0].strftime('%Y-%m-%d')]

    ms = 50
    total = {x: [0, 0] for x in stk_test.code.unique()}
    iso = {x: [0, 0] for x in stk_test.code.unique()}
    nis = {x: [0, 0] for x in stk_test.code.unique()}
    nis_s = {x: [0, 0] for x in stk_test.code.unique()}
    nis_c = {x: [0, 0] for x in stk_test.code.unique()}
    nis_b = {x: [0, 0] for x in stk_test.code.unique()}

    
    stk_test['td'] = stk_test.index
    stk_test.td = stk_test.td.diff()
    for _, v in stk_test[~(stk_test['td'] < datetime.timedelta(milliseconds=ms))].iterrows():
        if v['tick_type'] == 1:
            iso[v.code][0] += v['volume']
            total[v.code][0] += v['volume']
        elif v['tick_type'] == 2:
            iso[v.code][1] += v['volume']
            total[v.code][1] += v['volume']
    print(len(stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))
    for i, v in stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))].iterrows():
        time_package = [i - datetime.timedelta(milliseconds=ms), i + datetime.timedelta(milliseconds=ms)]
        nis_package = stk_test[(stk_test.index > time_package[0]) & (stk_test.index <= time_package[1])].groupby(['ts', 'code']).sum()
        if len(nis_package) == 1:
            if v['tick_type'] == 1: # 外盤成交
                iso[v.code][0] += v['volume']
                total[v.code][0] += v['volume']
            elif v['tick_type'] == 2: # 內盤成交
                iso[v.code][1] += v['volume']
                total[v.code][1] += v['volume']
        elif len(nis_package.index.get_level_values(1).unique()) == 1:
            if v['tick_type'] == 1: # 外盤成交
                nis_s[v.code][0] += v['volume']
                total[v.code][0] += v['volume']
                nis[v.code][0] += v['volume']
            elif v['tick_type'] == 2: # 內盤成交
                nis_s[v.code][1] += v['volume']
                total[v.code][1] += v['volume']
                nis[v.code][1] += v['volume']
        else:
            if len(nis_package[nis_package.index.get_level_values(1) == v['code']]) == 1:
                if v['tick_type'] == 1: # 外盤成交
                    nis_c[v.code][0] += v['volume']
                    total[v.code][0] += v['volume']
                    nis[v.code][0] += v['volume']
                elif v['tick_type'] == 2: # 內盤成交
                    nis_c[v.code][1] += v['volume']
                    total[v.code][1] += v['volume']
                    nis[v.code][1] += v['volume']
            else:
                if v['tick_type'] == 1: # 外盤成交
                    nis_b[v.code][0] += v['volume']
                    total[v.code][0] += v['volume']
                    nis[v.code][0] += v['volume']
                elif v['tick_type'] == 2: # 內盤成交
                    nis_b[v.code][1] += v['volume']
                    total[v.code][1] += v['volume']
                    nis[v.code][1] += v['volume']
    print(tick.split('.')[0])
    for k, l in {'total': total, 'iso': iso, 'nis': nis, 'nis_s': nis_s, 'nis_c': nis_c, 'nis_b': nis_b}.items():
        pf1 = pf.loc[l.keys()]
        pf1.name = 'returns'
        se = pd.Series({key: (v1[0]-v1[1]) / (v1[0]+v1[1]) if (v1[0] + v1[1]) != 0 else 0 for key, v1 in l.items()})
        se.name = 'coi'
        d = pd.concat([se, pf1], axis=1)#.corr().iloc[0, 1]
        d.sort_values('coi', ascending=False, inplace=True)
        d.reset_index(inplace=True)
        d = pd.concat([d, pd.Series([k]*len(d), name='Type')], axis=1)
        dh = d.iloc[:5, :].reset_index()
        dl = d.iloc[-5:, :].reset_index()
        dc = pd.concat([dh, dl], axis=1)
        df_ = pd.concat([df_, dc], axis=0)
        #globals()[k + '_'].append(d.corr().iloc[1, 2])
        #print(k, '{:.2f}'.format(d.corr().iloc[1, 2]))
    print('耗時: ', '{:.2f}'.format(time.time() - ctime))
    return df_
for tick in tick_list[-1:]:
    df_ = calculator(df_, tick)


27769
2022-11-07
耗時:  157.94


In [4]:
df_.T.to_csv(r'C:\Users\ChastLai\Documents\Python Scripts\Output\PaperTrade\COI.csv')

In [6]:
long = df_.iloc[:, :5].dropna(how='all')
short = df_.iloc[:, 5:].dropna(how='all')
#pd.concat([df_.iloc[:, :4].dropna(), df_.iloc[:, 4:].dropna()], axis=1, ignore_index=True)

In [7]:
print(long[long['Type'] == 'nis_s'])
print(short[short['Type'] == 'nis_s'])


   level_0  index       coi  returns   Type
0        0   3006  0.867403      NaN  nis_s
1        1   2027  0.859307      NaN  nis_s
2        2   2303  0.759374      NaN  nis_s
3        3   5425  0.715385      NaN  nis_s
4        4   1605  0.645374      NaN  nis_s
   level_0  index       coi  returns   Type
0       39   2376 -0.576733      NaN  nis_s
1       40   2409 -0.624288      NaN  nis_s
2       41   2610 -0.780183      NaN  nis_s
3       42   2317 -0.950862      NaN  nis_s
4       43   3008 -1.000000      NaN  nis_s


In [11]:
short

,Type,level_0,index,coi,returns,Type
0,total,40,2618,-0.216276,NaN,total
1,total,41,2317,-0.237689,NaN,total
2,total,42,2409,-0.268714,NaN,total
3,total,43,2610,-0.297063,NaN,total
4,total,44,2330,-0.329384,NaN,total
0,iso,40,2610,-0.197597,NaN,iso
1,iso,41,3035,-0.220094,NaN,iso
2,iso,42,2330,-0.266472,NaN,iso
3,iso,43,2409,-0.270144,NaN,iso
4,iso,44,2317,-0.275048,NaN,iso


In [84]:
long.columns = long.columns + '_long'
short.columns = short.columns + '_short'
long.reset_index(inplace=True)
short.reset_index(inplace=True)
ls = pd.concat([long, short], axis=1)
ls

,index,index_long,coi_long,returns_long,Type_long,index,index_short,coi_short,returns_short,Type_short
0,0.0,2201.0,0.161677,0.005734,total,44,2615.0,-0.413142,-0.063625,total
1,1.0,2409.0,0.160176,-0.008798,total,45,6278.0,-0.425154,-0.004902,total
2,2.0,6116.0,0.070666,-0.004167,total,46,2027.0,-0.443005,-0.015524,total
3,3.0,6182.0,0.064499,-0.009709,total,47,3189.0,-0.471202,0.000000,total
4,4.0,8069.0,0.057304,0.025424,total,48,3006.0,-0.504779,-0.020566,total
...,...,...,...,...,...,...,...,...,...,...
1105,2.0,3037.0,1.000000,-0.040486,nis_b,44,3035.0,-1.000000,-0.021978,nis_b
1106,3.0,2308.0,0.000000,0.003922,nis_b,45,1101.0,-1.000000,0.010000,nis_b
1107,4.0,8069.0,0.000000,0.027624,nis_b,46,6285.0,-1.000000,-0.002439,nis_b
1108,NaN,NaN,NaN,NaN,NaN,47,2603.0,-1.000000,0.007246,nis_b


In [116]:
for t in ['total', 'iso', 'nis', 'nis_s', 'nis_c', 'nis_b']:
    print(t, '{:.2f}'.format(ls[ls['Type_long'] == t].reset_index(drop=True).groupby(ls[ls['Type_long'] == t].reset_index(drop=True).index//5).sum().corr().iloc[3,2]))
    print(t, '{:.2f}'.format(ls[ls['Type_short'] == t].reset_index(drop=True).groupby(ls[ls['Type_short'] == t].reset_index(drop=True).index//5).sum().corr().iloc[-1,-2]))
    print('return: ', '{:.2f}'.format(ls[ls['Type_long'] == t].sum()['returns_long'] - ls[ls['Type_short'] == t].sum()['returns_short']))
#ls.groupby(ls.index//5).sum()

total 0.11
total 0.10
return:  -0.34
iso 0.08
iso 0.09
return:  -0.18
nis 0.30
nis 0.14
return:  -0.09
nis_s 0.44
nis_s 0.30
return:  0.44
nis_c -0.06
nis_c -0.11
return:  0.18
nis_b 0.09
nis_b 0.22
return:  0.09


C:\Users\ChastLai\AppData\Local\Temp\ipykernel_16288\3586739077.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('return: ', '{:.2f}'.format(ls[ls['Type_long'] == t].sum()['returns_long'] - ls[ls['Type_short'] == t].sum()['returns_short']))


In [114]:
for t in ['total', 'iso', 'nis', 'nis_s', 'nis_c', 'nis_b']:
    ls[ls['Type_long'] == 'nis_s'].sum()['returns_long'] - ls[ls['Type_short'] == 'nis_s'].sum()['returns_short']

0.44361125363971565

In [31]:
ms = 100
for tick in tick_list:
    stk_test = pd.read_json(path + tick)
    stk_test['ts'] = pd.to_datetime(stk_test['ts'])
    stk_test.set_index('ts', inplace=True)
    stk_test['td'] = stk_test.index
    stk_test.td = stk_test.td.diff()
    print(tick)
    print('iso_len: ', len(stk_test[~(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))
    print('nis_len: ', len(stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))

2022-09-01.json
iso_len:  192338
nis_len:  47572
2022-09-02.json
iso_len:  203630
nis_len:  53000
2022-09-05.json
iso_len:  159038
nis_len:  37177
2022-09-06.json
iso_len:  162272
nis_len:  33332
2022-09-07.json
iso_len:  153871
nis_len:  37283
2022-09-08.json
iso_len:  137825
nis_len:  32462
2022-09-12.json
iso_len:  147739
nis_len:  34889
2022-09-13.json
iso_len:  149964
nis_len:  36793
2022-09-14.json
iso_len:  172461
nis_len:  31745
2022-09-15.json
iso_len:  157423
nis_len:  30601
2022-09-16.json
iso_len:  150545
nis_len:  27736
2022-09-19.json
iso_len:  149136
nis_len:  29460
2022-09-20.json
iso_len:  164748
nis_len:  32943
2022-09-21.json
iso_len:  202163
nis_len:  79345
2022-09-22.json
iso_len:  197715
nis_len:  45887
2022-09-23.json
iso_len:  172529
nis_len:  37619
2022-09-26.json
iso_len:  194841
nis_len:  38171
2022-09-27.json
iso_len:  167639
nis_len:  34763
2022-09-28.json
iso_len:  206242
nis_len:  45836
2022-09-29.json
iso_len:  181904
nis_len:  43723
2022-09-30.json
iso_